<a href="https://colab.research.google.com/github/karasu1982/WebScraping/blob/main/WebScraping(NAVITIME).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境設定

In [ ]:
# 標準ライブラリ
import pandas as pd
import numpy as np
import datetime as dt
import time
import math

# スクレイピング用ライブラリ
import httplib2
from bs4 import BeautifulSoup as bs
import urllib.request

# 関数定義

In [ ]:
import re

# 文字列置換ロジック
r = re.compile( '(%s.*%s)' % ("\(","\)"), flags=re.DOTALL)

In [ ]:
def scrapingNAVITIME(list1, list2):

  base_url = "https://www.navitime.co.jp/category/{chain}/{pref}"
  headers = {'User-Agent':'Mozilla/5.0'}

  pref_list = []
  chain_list = []
  store_list = []
  address_list = []

  # class名
  ## 市区町村別URLを表すclass
  class_address_list="address-list-item-link"

  ## 店舗名・住所を表すclass
  class_spot_name = "spot-name" # 店舗名
  class_spot_detail = "spot-detail-section" # 店舗情報
  class_address = "spot-detail-value-text" # 店舗住所の内容（その中の１番目に住所がある）

  # 1ページの件数表示
  n_pages=15

  for l1, l2 in zip(list1, list2):
      for i in range(47):
      #for i in range(1):
          pref = '{0:02d}'.format(i+1)

          print(pref)
          url1 = []
          nums = []

          # 第一階層：市区町村URL取得
          print('第一階層：' + dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
          h = httplib2.Http()
          head, body = h.request(uri = base_url.format(chain=l1, pref = pref))
          soup = bs(body, 'html.parser')
          b = soup.body

          elems=b.find_all(class_=class_address_list)

          for elem in elems:
              try:
                  url = elem.get('href')
                  url1.append(url)

                  num = r.search(elem.text).group(0).replace("(","").replace(")","")
                  nums.append(num)
              except:
                  pass

          # 第二階層：店舗名・住所取得
          print('第二階層：' + dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
          timer1 = dt.datetime.now()
          counter = 0

          for u, num in zip(url1,nums):
              max_page=math.ceil(int(num)/n_pages)

              for i in range(max_page):
                  #print(u+"?page="+str(i+1))
                  h = httplib2.Http()
                  head, body = h.request(uri = u+"?page="+str(i+1), headers=headers)
                  soup = bs(body, 'html.parser')
                  b = soup.body

                  stores = b.find_all(class_=class_spot_name)

                  # 同じclass名で住所や電話番号が入っているため、各店舗の最初のclass名のセルのみを取得
                  addresses = b.select(f'dl[class="{class_spot_detail}"]')
                  addresses2 = [] 

                  for add in addresses:
                    addresses2.append(add.find(class_=class_address))
                    
                  if len(stores)!=len(addresses2):
                    print(l2+"は、店舗と住所がズレている")

                  for store, address in zip(stores, addresses2):
                    pref_list.append(pref)
                    chain_list.append(l2)
                    store_list.append(store.text.replace("\n","").replace("\t",""))
                    address_list.append(address.text.replace("\n",""))

                  time.sleep(1)

              timer2 = dt.datetime.now()
              counter += 1
              print('    ' + str(counter) + ' / ' + str(len(url1)) + ' : ' + str((timer2 - timer1).seconds) + 'sec')

  return store_list, address_list, pref_list, chain_list

# 処理実行

In [ ]:
# list1はchainのIDをリスト化、list2はその名称
list1 = ["0201001001","0201001009", "0201001006", "0201001005", "0201001004", "0201001011", "0201001003"]
list2 = ["セブンイレブン", "ローソン", "ファミリーマート", "ミニストップ", "デイリーヤマザキ", "セイコーマート", "その他コンビニ"]

store_list, address_list, pref_list, chain_list = scrapingNAVITIME(list1, list2)

df_output = pd.DataFrame({"store_name":store_list,"address":address_list,"pref":pref_list,"chain":chain_list})
df_output.head()